# Imports 

This Notebook is sequentially processing Images from SuperCaustics to be used in Cleargrasp Training. 
The Code currently does not check for many things that can go wrong during copy-pasting data, so definitely make backups from your images before running this. 

In [10]:
import os
import random as r
import shutil

import PIL.Image as Image
import cv2
import imageio
import numpy as np
from PIL import ImageChops
from PIL import ImageFilter

In [2]:
imageio.plugins.freeimage.download()

# Make Dataset Folders

In [3]:
#point this to the RAW Screenshots folder. Make Backups before running. You can Comment Out any files that you are not collecting.
os.chdir('D:/')
dir_src = ('D:/SuperCaustics/Saved/Screenshots/Windows/')

Depth = (dir_src + "Depth/")
Normal = (dir_src + "/Normal/")
Pic = (dir_src + "/Picture_No_Caustic/")
Caustic = (dir_src + "/Picture_Caustic/")
Caustic_Seg = (dir_src + "/Caustic_Seg/")
Outlines = (dir_src + "/Outlines/")
Outlines_Processed = (dir_src + "/Outlines_Processed/")
Seg = (dir_src + "/Seg/")
Seg_Processed = (dir_src + "/Seg_Processed/")
Depth_exr = (dir_src + "/Depth_exr/")
Normal_exr = (dir_src + "/Normal_exr/")
Seg_exr = (dir_src + "/Seg_exr/")

try:
    os.mkdir(Depth)
    os.mkdir(Normal)
    os.mkdir(Pic)
    os.mkdir(Caustic_Seg)
    os.mkdir(Caustic)
    os.mkdir(Outlines)
    os.mkdir(Seg)
    os.mkdir(Outlines_Processed)
    os.mkdir(Seg_Processed)
    os.mkdir(Depth_exr)
    os.mkdir(Normal_exr)
    os.mkdir(Seg_exr)

except OSError:
    print('Directory not created.')



# Move images into their respective folders (Order is important)

Use the Same order as you did when capturing your data from AIP SuperCaustics. In my case, I made screenshots in the following sequence caustic, no caustics, segmentation mask, normal map, depth map, outlines. 

In [ ]:
def move_file(filename, destination):
    shutil.move(dir_src + filename, destination)
    print("moved " + filename, end='\r')

In [7]:
caustic = 1
counter = 0
destinations = [Caustic, Pic, Seg, Normal, Depth, Outlines]

for filename in os.listdir(dir_src):
    if not filename.endswith('.png'):
        continue

    move_file(filename, destinations[counter])
    counter = (counter + 1) % len(destinations)

# Rename raw images in each folder to have the same name

In [8]:
def rename_files(directory):
    i = 0
    for filename in os.listdir(directory):
        os.rename(directory + '/' + filename, directory + '/' + str(i) + '.png')
        i += 1
        print("renamed " + filename, end='\r')

In [9]:
directories = [Pic, Caustic, Depth, Normal, Outlines, Seg]

for directory in directories:
    rename_files(directory)

# Image compression

Segmantation masks and ourlines by their nature store data with a limited set of colors (most ofthen with just black and white), so we can convert them to greyscale images, so they take up 3+ times less space in memory.

In [11]:
for filename in os.listdir(Seg):
    segaddress = Seg + filename
    newaddress = Seg_Processed + filename
    img = []
    arr = np.array(Image.open(segaddress).convert('L'))
    img = np.where(arr >= 220, 255, -1)
    cv2.imwrite(newaddress, img)
    print(filename + 'Process'
                     'ed.', end='\r')

print('Seg Files Processed.')

for filename in os.listdir(Outlines):
    outaddress = Outlines + filename
    newaddress = Outlines_Processed + filename
    image = Image.open(outaddress)
    image = image.convert('L')
    arr = np.asarray(image)
    arr = np.floor(arr / 126)
    cv2.imwrite(newaddress, arr)
    print(filename + 'Processed.', end='\r')

print('Outline Files Processed.')

Seg Files Processed.
Outline Files Processed.rocessed.406.pngProcessed.412.pngProcessed.42.pngProcessed.426.pngProcessed.432.pngProcessed.439.pngProcessed.445.pngProcessed.452.pngProcessed.46.pngProcessed.466.pngProcessed.473.pngProcessed.48.pngProcessed.485.pngProcessed.492.pngProcessed.499.pngProcessed.503.pngProcessed.510.pngProcessed.516.pngProcessed.521.pngProcessed.527.pngProcessed.533.pngProcessed.54.pngProcessed.546.pngProcessed.551.pngProcessed.555.pngProcessed.560.pngProcessed.567.pngProcessed.573.pngProcessed.58.pngProcessed.585.pngProcessed.590.pngProcessed.597.pngProcessed.601.pngProcessed.607.pngProcessed.613.pngProcessed.62.pngProcessed.624.pngProcessed.629.pngProcessed.634.pngProcessed.64.pngProcessed.645.pngProcessed.651.pngProcessed.658.pngProcessed.662.pngProcessed.668.pngProcessed.673.pngProcessed.679.pngProcessed.684.pngProcessed.690.pngProcessed.695.pngProcessed.700.pngProcessed.707.pngProcessed.712.pngProcessed.718.pngProcessed.722.pngProcessed.728.pngProcessed.7

## Processing Caustic Segmentation

In order to train a model to recognize caustics, we first need 

In [12]:
def extract_caustic_segmentation(path_img_with_caustic, path_img_no_caustic, path_segmentation, thresh=10):
    import PIL.Image as Image
    # Binary Function
    fn = lambda x: 255 if x > thresh else 0

    img_with_caustic = Image.open(path_img_with_caustic).convert('RGB')
    img_no_caustic = Image.open(path_img_no_caustic).convert('RGB')
    # Convert image to a binary black/white image
    img_segmentation = Image.open(path_segmentation).convert('1')

    # Perform a per-pixel subtraction, and because caustics have mostly specular lighting
    # the pixels which are illuminated by them will have greater luminance, and hence a
    # greater sum of their RGB values than its non-caustic counterpart. Which is exactly
    # how we'll extract them
    acc_image = ImageChops.subtract(img_with_caustic, img_no_caustic)

    # Make Binary Image
    r = acc_image.convert('L').point(fn, mode='1')

    # Remove Noise
    p = ImageChops.subtract(r, img_segmentation)
    p = p.filter(ImageFilter.MinFilter(5))
    p = p.filter(ImageFilter.MaxFilter(5))

    return p

In [13]:
for filename in os.listdir(Seg_Processed):
    Seg = str(Seg_Processed + filename)
    Image = str(Caustic + filename)
    NoCaustic = str(Pic + filename)
    extract_caustic_segmentation(Image, NoCaustic, Seg).save(Caustic_Seg + filename)
    print(filename + 'Processed.', end='\r')

# Choosing a Random Validation Set: Making the Folders

In [14]:
dir_val = ('D:/Dataset/Dataset/ExampleData_val/')

valDepth = (dir_val + "/Depth/")
valNormal = (dir_val + "/Normal/")
valPic = (dir_val + "/Picture_No_Caustic/")
valCaustic = (dir_val + "/Picture_Caustic/")
valOutlines = (dir_val + "/Outlines/")
valOutlines_Processed = (dir_val + "/Outlines_Processed/")
valSeg = (dir_val + "/Seg/")
valSeg_Processed = (dir_val + "/Seg_Processed/")
valDepth_exr = (dir_val + "/Depth_exr/")
valNormal_exr = (dir_val + "/Normal_exr/")
valSeg_exr = (dir_val + "/Seg_exr/")

try:
    os.mkdir(dir_val)
    os.mkdir(valDepth)
    os.mkdir(valNormal)
    os.mkdir(valPic)
    os.mkdir(valCaustic)
    os.mkdir(valOutlines)
    os.mkdir(valSeg)
    os.mkdir(valOutlines_Processed)
    os.mkdir(valSeg_Processed)
    os.mkdir(valDepth_exr)
    os.mkdir(valNormal_exr)
    os.mkdir(valSeg_exr)

except OSError:
    print('Directory not created.')



# Moving The Processed Files to Validation Set

In [64]:
Depth = Depth.replace('//', '/')[:-1]
Normal = Normal.replace('//', '/')[:-1]
Pic = Pic.replace('//', '/')[:-1]
Caustic = Caustic.replace('//', '/')[:-1]
Outlines = Outlines.replace('//', '/')[:-1]
Outlines_Processed = Outlines_Processed.replace('//', '/')[:-1]
Seg = os.path.dirname(Seg.replace('//', '/'))
Seg_Processed = Seg_Processed.replace('//', '/')[:-1]
Depth_exr = Depth_exr.replace('//', '/')[:-1]
Normal_exr = Normal_exr.replace('//', '/')[:-1]
Seg_exr = Seg_exr.replace('//', '/')[:-1]

In [65]:
Seg_exr

'D:/SuperCaustics/Saved/Screenshots/Windows/Seg_exr'

In [70]:
valcount = 0
validationSetSize = 6

for filename in os.listdir(Pic):
    if valcount < validationSetSize and filename.endswith('.png') and r.randint(0, 6) == 6:
        shutil.move(Depth + '/' + filename, valDepth + '/' + filename)
        shutil.move(Normal + '/' + filename, valNormal + '/' + filename)
        shutil.move(Pic + '/' + filename, valPic + '/' + filename)
        shutil.move(Caustic + '/' + filename, valCaustic + '/' + filename)
        shutil.move(Outlines + '/' + filename, valOutlines + '/' + filename)
        shutil.move(Outlines_Processed + '/' + filename, valOutlines_Processed + '/' + filename)
        shutil.move(Seg + '/' + filename, valSeg + '/' + filename)
        shutil.move(Seg_Processed + '/' + filename, valSeg_Processed + '/' + filename)
        # shutil.copy(Depth_exr + '/' + filename.replace(".png", ".exr"), valDepth_exr + '/' + filename.replace(".png", ".exr"))
        # shutil.copy(Normal_exr + '/' + filename.replace(".png", ".exr"), valNormal_exr + '/' + filename.replace(".png", ".exr"))
        # shutil.copy(Seg_exr + '/' + filename.replace(".png", ".exr"), valSeg_exr + '/' + filename.replace(".png", ".exr"))

        print("Chosen " + filename, end='\r')
        valcount += 1

FileNotFoundError: [WinError 2] The system cannot find the file specified